In [ ]:
import pandas as pd
import numpy as np
import pymongo as pymongo
import json
import pprint

In [ ]:
###
#Prerequisitos para el File Dialog

from tkinter import*
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

from tkinter import filedialog

### Leer archivo excel

In [ ]:
####
#File dialog para seleccionar el archivo "RESULTADOS APP PARA INGENIERIA - AN.XLSX"

infiles = filedialog.askopenfilename(multiple=True)

%gui tk

In [ ]:
#Cargar archivo de muestras
df = pd.ExcelFile(infiles[0])
df1 = df.parse('pruebas por experimento')
df1.head()

### Validar Nombres de columnas 

In [ ]:
#Validar nombres de columnas para carga a MongoDB
col = list(df1)

In [ ]:
l= []
for n in col:
    l.append(n.lower().replace(" ", "_"))
l    

In [ ]:
df1.columns = l
df1.head()

### Validamos que el QR no se encuentre en BD
####          Esto lo logramos extrayendo la lista d QR's en la BD y revisando si ISIN

In [ ]:
#Establece conexion 1)nombre de la base datos
client = pymongo.MongoClient("idenmon.zapto.org", 888)
db = client.prodLaboratorio
pruebas = db.resultsExperiments

In [ ]:
pruebas_db = pruebas.find()
pruebas_list = list (pruebas_db)
row_preinsert = len (pruebas_list)
row_preinsert

In [ ]:
qr=[]
for n in range(0,len(pruebas_list)):
    qr.append(pruebas_list[n]['qr'])

In [ ]:
dfRed = pd.DataFrame
dfRed = df1.drop(df1[df1['qr'].isin(qr)].index)
dfRed

### Convertir a JSON para poder almacenar en mongoDB

In [ ]:
#Utilizando conversion directa a una lista de diccionarios en python
s = dfRed.to_json(orient = 'records')
l = json.loads(s)

In [ ]:
l

### Guardar archivos en mongoDB

In [ ]:
#Contar numero de registros con un checksum
num_rows_input = len(l)
num_rows_input

In [ ]:
#Insertar JSON a mongoDB
if l:
    pruebas.insert_many(l)
else:
    print('No hay datos para guardar')

In [ ]:
#desplegar en python df
pruebas_db = pruebas.find()
pruebas_list = list (pruebas_db)
dfp = pd.DataFrame(pruebas_list)
dfp.head()

In [ ]:
#Confirmar escritura
#num_rows_input
num_rows_output = len (pruebas_list)
#num_rows_output
sum_rows_input = (row_preinsert + num_rows_input)
ok_flag = (num_rows_output == sum_rows_input)
#ok flag
#num_rows_output
ok_flag